In [449]:
from ipynb.fs.full.data_extraction import init_spark, read_file
from pyspark.sql.functions import count,when,isnan,dayofmonth, month, year,col,udf,struct
from pyspark.sql.types import IntegerType,LongType

In [450]:
spark = init_spark()

In [451]:
match = read_file("Match.csv")
player_attributes = read_file("Player_Attributes.csv")

In [452]:
match.filter(match["home_player_1"].isNotNull()).select("home_player_1","date")
player_attributes.filter(player_attributes["player_api_id"] == 39890).select("player_api_id","overall_rating").show()

+-------------+--------------+
|player_api_id|overall_rating|
+-------------+--------------+
|        39890|            64|
|        39890|            66|
|        39890|            66|
|        39890|            68|
|        39890|            67|
|        39890|            67|
|        39890|            66|
|        39890|            64|
|        39890|            62|
|        39890|            62|
+-------------+--------------+



In [453]:
# print(match.columns)
# match.select("home_player_X1").filter(match["home_player_X1"].isNotNull()).count()
required_columns = ["match_api_id","date","home_team_goal","away_team_goal","home_player_1", "home_player_2", "home_player_3", "home_player_4", "home_player_5",
               "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
               "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
               "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11","B365H", "B365D", "B365A", "BWH", "BWD", "BWA"]

match = match.select(required_columns)

In [454]:
match = match.na.drop()
player_attributes = player_attributes.na.drop()

In [455]:
def get_overall_rating(row,match_df_columns):
    match_id = row[match_df_columns.index("match_api_id")]
    match_date = row[match_df_columns.index("date")]
    players = ['home_player_1', 'home_player_2', 'home_player_3', "home_player_4", "home_player_5",
               "home_player_6", "home_player_7", "home_player_8", "home_player_9", "home_player_10",
               "home_player_11", "away_player_1", "away_player_2", "away_player_3", "away_player_4",
               "away_player_5", "away_player_6", "away_player_7", "away_player_8", "away_player_9",
               "away_player_10", "away_player_11"]
    for player in players:
        player_id = row[match_df_columns.index(player)]
        attributes = player_attributes.filter(player_attributes["player_api_id"] == player_id)
        current_attributes = attributes.filter(attributes["date"]<match_date).orderBy("date",ascending=False)
        overall_rating = current_attributes.select("overall_rating").collect()
        ratings = []
        for rating in overall_rating:
            ratings.append(rating.overall_rating)
        print(ratings)
        return sum(ratings)/len(ratings)

# overall_rating_udf = udf(get_overall_rating)

In [456]:
required_columns = required_columns + ["overall_rating","player_api_id"]

In [457]:
match_data = match.join(player_attributes, on = "date").select(required_columns)

In [458]:
player_attributes.filter((year(player_attributes["date"]) == 2009) & (month(player_attributes["date"]) == 8) 
                                                                    & (dayofmonth(player_attributes["date"] ) == 30))\
.filter(player_attributes["player_api_id"] == 39890).select("date","overall_rating","player_api_id").show()

+-------------------+--------------+-------------+
|               date|overall_rating|player_api_id|
+-------------------+--------------+-------------+
|2009-08-30 00:00:00|            67|        39890|
+-------------------+--------------+-------------+



In [468]:
match_data.filter((year(match["date"]) == 2009) & (month(match["date"]) == 8) 
                                                                    & (dayofmonth(match["date"] ) == 30))\
.groupBy(["date","player_api_id"]).mean().select("player_api_id","avg(overall_rating)")

DataFrame[player_api_id: int, avg(overall_rating): double]

+-------------------+------------------+------------------+
|               date|avg(home_player_1)|avg(home_player_2)|
+-------------------+------------------+------------------+
|2015-03-20 00:00:00|           68813.0|          195335.0|
|2015-04-17 00:00:00|          120621.0|          361320.0|
|2013-04-26 00:00:00|           32679.0|          112704.0|
|2013-12-20 00:00:00|           37579.0|          195335.0|
|2015-05-01 00:00:00|          187563.0|          210429.0|
|2014-12-19 00:00:00|           73462.0|          243465.0|
|2013-05-03 00:00:00|          301804.0|          210423.0|
|2013-03-15 00:00:00|           30820.0|           20446.0|
|2013-10-04 00:00:00|           41087.0|          116645.0|
|2016-02-13 00:00:00|           32562.0|           31701.0|
|2015-04-17 00:00:00|          145550.0|          167094.0|
|2009-02-22 00:00:00|           27558.0|           27566.0|
|2013-03-15 00:00:00|           37780.0|          181227.0|
|2008-08-30 00:00:00|           27504.0|